In [50]:
import  numpy
import  pandas
import   tqdm
# !pip install scikit_learn
# !pip install catboost

In [51]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from catboost import CatBoostRegressor

pd.set_option('display.max_columns', 200)

In [52]:
# from google.colab import drive
# drive.mount('/content/drive')
X = pd.read_csv('/content/X.csv')
y = pd.read_csv('/content/y.csv')

In [53]:
# X = pd.read_csv("/content/drive/MyDrive/S7/X.csv")
# y = pd.read_csv("/content/drive/MyDrive/S7/y.csv")

In [54]:
merged = X.merge(y)

In [55]:
#Merged Data
Mt5b = merged[merged['flight_phase'] == 'TAKEOFF']
Mt5b = Mt5b[Mt5b['engine_family'] == 'CFM56-5B']

Mc5b = merged[merged['flight_phase'] == 'CRUISE']
Mc5b = Mc5b[Mc5b['engine_family'] == 'CFM56-5B']

Mt8e = merged[merged['flight_phase'] == 'TAKEOFF']
Mt8e = Mt8e[Mt8e['engine_family'] == 'CF34-8E']

Mc8e = merged[merged['flight_phase'] == 'CRUISE']
Mc8e = Mc8e[Mc8e['engine_family'] == 'CF34-8E']

Mt7 = merged[merged['flight_phase'] == 'TAKEOFF']
Mt7 = Mt7[Mt7['engine_family'] == 'CFM56-7']

Mc7 = merged[merged['flight_phase'] == 'CRUISE']
Mc7 = Mc7[Mc7['engine_family'] == 'CFM56-7']

#Split for X and y for engine_type and flight_phase

col = [0, 2, 3]
for i in range(53, len(Mc7.columns)):
  col.append(i)

Xt5b = Mt5b.iloc[:,:-30]
yt5b = Mt5b.iloc[:,col]

Xc5b = Mc5b.iloc[:,:-30]
yc5b = Mc5b.iloc[:,col]

Xt8e = Mt8e.iloc[:,:-30]
yt8e = Mt8e.iloc[:,col]

Xc8e = Mc8e.iloc[:,:-30]
yc8e = Mc8e.iloc[:,col]

Xt7 = Mt7.iloc[:,:-30]
yt7 = Mt7.iloc[:,col]

Xc7 = Mc7.iloc[:,:-30]
yc7 = Mc7.iloc[:,col]


In [56]:
Xnames = ['Xt5b', 'Xc5b', 'Xt8e', 'Xc8e', 'Xt7', 'Xc7']
ynames = ['yt5b', 'yc5b', 'yt8e', 'yc8e', 'yt7', 'yc7']


In [57]:
def get_score_by_parameter(predict_field, XX, yy):
    df = pd.merge(XX, yy, on=["engine_id", "flight_datetime", "flight_phase"])
    df = df[df[predict_field].notna()]
    df = df.fillna(-100)
    X = df.drop(columns=[predict_field])
    X = X.drop(columns=[
        "engine_id", "aircraft_id", "flight_datetime",
        "flight_phase", "engine_family", "engine_type", "manufacturer",
        "aircraft_family", "aircraft_type", "aircraft_grp", "ac_manufacturer"])
    y = df[predict_field]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    
    model = CatBoostRegressor(iterations=500, verbose=False)
    model.fit(X_train, y_train)
    catboost_predicted = model.predict(X_test)
    catboost_score = mean_squared_error(y_test, catboost_predicted, squared=False)
    catboost_predicted = model.predict(X)
    return catboost_score


for i in range(6):
  exec(f"{Xnames[i]} = {Xnames[i]}.drop(columns=['ZHPTAC', 'ZLPTAC' , 'ZPHSF' , 'ZPN12R' ,'ZPOIL', 'ZPS3', 'ZT1AB' ,'ZT3', 'ZTAMB', 'ZTLA', 'ZTNAC', 'ZTOIL', 'ZVSV', 'ZWF36', 'IHPSOV', 'SAT', 'ZVIAS',  'ZWBP1_8E', 'ZWBP2_8E', 'IBP'])")
  print(Xnames[i])
  scores = pd.DataFrame(columns=["parameter", "rmse"])
  exec(f"output_parameters = {ynames[i]}.columns.drop(['flight_datetime', 'flight_phase', 'engine_id'])")

  for param in tqdm(output_parameters):
      try:
          exec(f"score = get_score_by_parameter(param, {Xnames[i]}, {ynames[i]})")
          #qua = df[param].quantile([.0, .1, .5, .9, 1])
          #qua = list(qua)
          scores.loc[len(scores)] = [param, score] 
      except:
          continue
  print(scores)

Xt5b


100%|██████████| 30/30 [00:55<00:00,  1.85s/it]


   parameter      rmse
0       BRAT  0.017784
1      DELFN  0.151761
2      DELN1  0.063353
3     EGTHDM  1.433408
4   EGTHDM_D  0.174432
5      PCN12  0.084798
6     PCN12I  0.081705
7     PCN1AR  0.130459
8     PCN1BR  0.339564
9      PCN1K  0.105355
10    SLOATL  0.457827
11  SLOATL_D  0.052444
12       WBE  0.609650
13  ZPCN25_D  0.077295
14    ZT49_D  0.955837
Xc5b


100%|██████████| 30/30 [02:25<00:00,  4.84s/it]


   parameter       rmse
0       BRAT   0.004977
1       DEGT   0.699499
2     DELVSV   0.119783
3      DPOIL   0.662545
4       EGTC   0.561368
5     EGTHDM   2.125425
6     GEGTMC   0.767370
7      GN2MC   0.013226
8     GPCN25   0.013384
9       GWFM   0.288243
10     PCN12   0.040996
11    PCN12I   0.040996
12     PCN1K   0.083396
13     PCN2C   0.026230
14    SLOATL   0.749564
15       WBE   0.327971
16       WBI   0.002686
17      WFMP  10.811444
18  ZPCN25_D   0.039630
19    ZT49_D   1.269483
20   ZTNAC_D   2.370136
21   ZWF36_D   5.719659
Xt8e


...

   parameter      rmse
0       BRAT  0.001445
1       DEGT  2.374823
2       EGTC  0.973790
3     GPCN25  0.074511
4       GWFM  0.621697
5      PCN12  0.128735
6     PCN12I  0.120599
7      PCN1K  0.107862
8      PCN2C  0.037126
9        WBI  0.001445
10      WFMP  7.538440
11  ZPCN25_D  0.058092
12    ZT49_D  3.938801
13    ZTLA_D  0.104947
14   ZWF36_D  9.913073
Xt7


100%|██████████| 30/30 [01:11<00:00,  2.39s/it]


   parameter      rmse
0       BRAT  0.053365
1      DELFN  0.119797
2      DELN1  0.091349
3     EGTHDM  0.400429
4   EGTHDM_D  0.175941
5      PCN12  0.065016
6     PCN12I  0.055158
7     PCN1AR  0.069326
8     PCN1BR  0.083793
9      PCN1K  0.059009
10    SLOATL  0.122084
11  SLOATL_D  0.046598
12  ZPCN25_D  0.603726
13    ZT49_D  2.304713
Xc7


100%|██████████| 30/30 [01:39<00:00,  3.33s/it]

   parameter        rmse
0       BRAT    0.011446
1       DEGT    0.260308
2      DPOIL    0.887415
3       EGTC    0.576304
4     EGTHDM    0.495553
5     GEGTMC    0.250384
6      GN2MC    0.007537
7     GPCN25    0.007473
8       GWFM    0.372021
9      PCN12    0.027029
10    PCN12I    0.027029
11     PCN1K    0.038110
12     PCN2C    0.026120
13    SLOATL    0.154790
14       WBI    0.012187
15      WFMP    9.294164
16  ZPCN25_D    0.919402
17    ZT49_D   14.862007
18   ZWF36_D  356.871353


In [58]:
# Xt5b.info()

In [59]:
# scores = pd.DataFrame(columns=["parameter", "rmse"])
# output_parameters = yt5b.columns.drop(['flight_datetime', 'flight_phase', 'engine_id'])

# for param in tqdm(output_parameters):
#     try:
#         score = get_score_by_parameter(param)
#         #qua = df[param].quantile([.0, .1, .5, .9, 1])
#         #qua = list(qua)
#         scores.loc[len(scores)] = [param, score] 
#     except:
#         continue

In [60]:
# scores